In [52]:
%load_ext autoreload
%autoreload 2

import numpy as np 
import onnxruntime as ort
import onnx
import datasets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from torchvision import transforms
import torch

def t_cropflip_augment_apply(examples):
    examples['img'] = [t_cropflip_augment(image) for image in examples['img']]
    return examples
    
def t_normalize_apply(examples):
    examples['img'] = [t_normalize(image) for image in examples['img']]
    return examples

# Also make them float
t_cropflip_augment = transforms.Compose([
    # take_img,
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

t_normalize = transforms.Compose([
    # take_img,
    transforms.ToTensor(),
    # transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [56]:
train_set = datasets.load_dataset("uoft-cs/cifar10", split="train")
test_set = datasets.load_dataset("uoft-cs/cifar10", split="test")

train_set.set_transform(t_cropflip_augment_apply)
test_set.set_transform(t_normalize_apply)

In [61]:
train_set[0]

{'img': tensor([[[-2.4291, -2.4291, -2.4291,  ...,  1.0408,  1.0408,  1.0214],
          [-2.4291, -2.4291, -2.4291,  ...,  1.0796,  1.0602,  1.0602],
          [-2.4291, -2.4291, -2.4291,  ...,  1.0214,  1.0214,  1.0214],
          ...,
          [-2.4291, -2.4291, -2.4291,  ..., -0.1223, -0.1223, -0.1804],
          [-2.4291, -2.4291, -2.4291,  ..., -0.2192, -0.1998, -0.2386],
          [-2.4291, -2.4291, -2.4291,  ..., -0.4906, -0.4712, -0.5100]],
 
         [[-2.4183, -2.4183, -2.4183,  ...,  1.0628,  1.0628,  1.0431],
          [-2.4183, -2.4183, -2.4183,  ...,  1.1021,  1.1021,  1.0824],
          [-2.4183, -2.4183, -2.4183,  ...,  1.0431,  1.0431,  1.0431],
          ...,
          [-2.4183, -2.4183, -2.4183,  ..., -0.1959, -0.2156, -0.2549],
          [-2.4183, -2.4183, -2.4183,  ..., -0.3729, -0.3729, -0.3926],
          [-2.4183, -2.4183, -2.4183,  ..., -0.5696, -0.5892, -0.5696]],
 
         [[-2.2214, -2.2214, -2.2214,  ...,  1.4661,  1.4856,  1.4661],
          [-2.2214, -

In [69]:
model(train_set[0]['img'].to(device).unsqueeze(0))

tensor([[ 0.0129, -0.0098,  0.0177,  0.0182,  0.0055, -0.0097,  0.0260, -0.0277,
          0.0244,  0.0026]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [79]:
import torch
from models import mbv2

def sup_train(model, train_loader, test_loader, test_set, optimizer, criterion, learning_epochs):
    for epoch in range(learning_epochs):
        model.train()
        epochscore = 0
        runloss = 0
        for indic in test_loader:
            model.train()
            inputs = indic["img"].to(device)
            labels = indic["label"].to(device)
            
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs,labels)

            loss.backward()
            optimizer.step()

            runloss += loss
            
            _, preds = torch.max(outputs,1)
            epochscore += torch.sum(preds == labels.data)

            model.eval()
            
        with torch.no_grad():
            _,accuracy = test(model, test_loader, criterion, test_set)
            print("Epoch: ", epoch, " Loss: ", runloss, " Accuracy: ", accuracy)

def test(model, test_loader, criterion, test_set):       
    ''' Tests the accuracy of the dp model on testset ''' 
    model.eval()
    loss = 0
    total_correct=0
    for indic in test_loader:
        inputs = indic["img"].to(device)
        labels = indic["label"].to(device)
        outputs = model(inputs)
        loss += criterion(outputs, labels).item()
        _, preds = torch.max(outputs,1)
        total_correct += torch.sum(preds == labels.data)
    accuracy = total_correct/len(test_set)
    return loss,accuracy



learning_epochs = 100
batch_size = 64

model = mbv2.MobileNetV2(
    num_classes=10,
    width_mult=1.0
).to(device)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False
)

sup_train(
    model=model,
    train_loader=train_loader,
    optimizer=torch.optim.Adam(model.parameters(), lr=0.001),
    criterion=torch.nn.CrossEntropyLoss(),
    learning_epochs=learning_epochs,
    test_loader=test_loader,
    test_set = test_set
)


Epoch:  0  Loss:  tensor(300.7662, device='cuda:0', grad_fn=<AddBackward0>)  Accuracy:  tensor(0.3930, device='cuda:0')
Epoch:  1  Loss:  tensor(250.1091, device='cuda:0', grad_fn=<AddBackward0>)  Accuracy:  tensor(0.4981, device='cuda:0')
Epoch:  2  Loss:  tensor(226.3119, device='cuda:0', grad_fn=<AddBackward0>)  Accuracy:  tensor(0.4942, device='cuda:0')
Epoch:  3  Loss:  tensor(207.7799, device='cuda:0', grad_fn=<AddBackward0>)  Accuracy:  tensor(0.5307, device='cuda:0')
Epoch:  4  Loss:  tensor(194.2081, device='cuda:0', grad_fn=<AddBackward0>)  Accuracy:  tensor(0.5351, device='cuda:0')
Epoch:  5  Loss:  tensor(178.2174, device='cuda:0', grad_fn=<AddBackward0>)  Accuracy:  tensor(0.5880, device='cuda:0')
Epoch:  6  Loss:  tensor(165.9576, device='cuda:0', grad_fn=<AddBackward0>)  Accuracy:  tensor(0.6342, device='cuda:0')
Epoch:  7  Loss:  tensor(152.4278, device='cuda:0', grad_fn=<AddBackward0>)  Accuracy:  tensor(0.6535, device='cuda:0')
Epoch:  8  Loss:  tensor(138.1372, devic

KeyboardInterrupt: 

In [81]:
test(model, test_loader, torch.nn.CrossEntropyLoss(), test_set)

(28.13067110395059, tensor(0.9410, device='cuda:0'))

In [82]:
torch.save(model.state_dict(), "saved_models/mbv2_cifar10.pt")

In [91]:
example_input = test_set[0]['img'].unsqueeze(0).to(device)
torch.onnx.export(model, example_input, "onnx_models/mbv2_cifar10.onnx")